Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

Загрузите стандартный датасет об алмазах из библиотеки Seaborn:

In [2]:
import seaborn as sns
import ssl
ssl._create_default_https_context=ssl._create_unverified_context
df = sns.load_dataset('diamonds')

In [3]:
# Удалите часть признаков:

df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [5]:
#Закодируйте категориальные признаки:
import pandas as pd
df = pd.get_dummies(df, drop_first=True)

In [7]:
#Логарифмируйте признаки:
import numpy as np
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [8]:
#Определите целевую переменную и предикторы:

X = df.drop(columns="price")
y = df["price"]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

"loss": ["squared_error", "epsilon_insensitive"],\
"penalty": ["elasticnet"],\
"alpha": np.logspace(-3, 3, 10),\
"l1_ratio": np.linspace(0, 1, 10),\
"learning_rate": ["constant"],\
"eta0": np.logspace(-4, -1, 4)

Обучите регрессию с оптимальными параметрами. В качестве ответа введите получившееся значение MSE, предварительно округлив его до третьего знака после точки-разделителя.

In [9]:

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

parameters = {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 10),
    "l1_ratio": np.linspace(0, 1, 10),
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
}

sgd = SGDRegressor(random_state=42)
sgd_cv = GridSearchCV(estimator=sgd, param_grid=parameters, n_jobs=-1)
sgd_cv.fit(X_train, y_train)

print(sgd_cv.best_params_)

sgd = SGDRegressor(**sgd_cv.best_params_, random_state = 42)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)

round(mean_squared_error(y_test, ls), 3)

{'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


0.044

## Задание 3.1

In [11]:
def func1(x):
    return 6*x**5-5*x**4-4*x**3+3*x**2
 
def func2(x):
    return 30*x**4-20*x**3-12*x**2+6*x

init_value = 0.7
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
    
print(iter_count)

0.6296335078534031
0.6286680781673306
0.6286669787778999
0.6286669787764609
4


In [12]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [13]:
#Можно объединить всё в одну функцию:
    
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [14]:
# А можно воспользоваться реализацией из библиотеки scipy:

from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [19]:
def func1(x):
    return 24*x**2 - 4*x
def func2(x):
    return 48*x - 4
newton(func=func1, fprime=func2, x0=42, tol=0.0001)

0.1666666807529666

# BFGS

In [20]:
import numpy as np
from scipy.optimize import minimize

# Определим функцию, которую будем оптимизировать. Вместо отдельных x и y можно взять координаты единого вектора:

def func(x):
    return x[0]**2.0 + x[1]**2.0

# Теперь определим градиент для функции:

def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

In [21]:
# Зададим начальную точку:
x_0 = [1.0, 1.0]

# Определим алгоритм:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

# Выведем результаты:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы получили, что минимум функции достигается в точке (0, 0). Значение функции в этой точке также равно нулю. 

Можно повторить то же самое с вариацией  L-BFGS-B:

In [22]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Результат будет тем же самым.

→ Иногда количество итераций у двух модификаций различается, но ответ совпадает. Бывает также, что одна из вариаций может не сойтись, а другая — достичь экстремума, поэтому советуем не воспринимать их как взаимозаменяемые алгоритмы. На практике лучше пробовать разные варианты: если у вас не сошёлся алгоритм BFGS, можно попробовать L-BFGS-B, и наоборот. Также можно экспериментировать одновременно с обоими алгоритмами, чтобы выбрать тот, который будет сходиться для функции за меньшее число итераций и тем самым экономить время.

→ Важно понимать, что для некоторых функций не из всех стартовых точек получается достичь сходимости метода. Тогда их можно перебирать, к примеру, с помощью цикла.

In [23]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])
 
# определяем начальную точку
x_0 = [-400, -400]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 9
Решение: f([-3.99999972  1.00000028]) = -1.00000


In [27]:
# определяем нашу функцию
def func(x):
    return x[0]**4 + 6*x[1]**2 + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000


### Пример № 1. SciPy (scipy.optimize.linprog)

In [29]:
import numpy as np
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

Здесь нам необходимо вспомнить линейную алгебру, так как очень важно, чтобы векторы были в нужных нам размерностях, иначе мы не сможем использовать матричное умножение. Вектор A размера 6 мы превращаем в матрицу размера (1, 6) с помощью функции expand_dims(). Создаём все необходимые переменные:

In [30]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [34]:
#Передаём подготовленные переменные в оптимизатор SciPy:

from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

### Пример № 2. CVXPY

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY.

In [ ]:
import cvxpy

In [ ]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

In [ ]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

### Пример № 3. PuLP

В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней —  автомобиль B.
Каждый специалист может работать суммарно 30 дней.

Целевая функция будет выглядеть следующим образом: 20000*A + 45000*B

Также запишем ограничения:
- A, B >= 0
- 4A + 5B <= 30
- 3A + 6B <= 30
- 2A + 7B <= 30

Заметьте, что здесь мы снова пишем обычные неравенства, а не условия в матричном виде. Дело в том, что для данной библиотеки так «удобнее», так как она принимает все условия в «первичном» виде.








In [1]:
from pulp import *
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jr/y6j710014w98cjhsh00zh_gw0000gn/T/1ae026a56de24f48989ec8bbe93a23b8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jr/y6j710014w98cjhsh00zh_gw0000gn/T/1ae026a56de24f48989ec8bbe93a23b8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 21 RHS
At line 25 BOUNDS
At line 28 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 216667 - 0.00 seconds
Cgl0004I processed model has 3 rows, 2 columns (2 integer (0 of which binary)) and 6 elements
Cutoff increment increased from 1e-05 to 5000
Cbc0012I Integer solution of -195000 found by DiveCoefficient after 0 iterations and 0 nodes (0.0